In [1]:
from ColonyMgmt import * 

In [2]:
# to do items
# add logging function; if colony is over 200, prioritize saccing and consolidating cages

## some fancy to dos:
### check that the csv is within an hour, i.e. up to date
### wrap helper functions more cleanly?
### eventually package as a script proper
### function to search the **NOTES** column; i.e. for mice that have bred with other mice etc

## some fiddling to make sure we are not pulling from the wrong file
# file = "05227_mastersheet - Active.tsv"

# def checkTime(path):
#     loc = os.getcwd() + "\\" + path
#     return os.path.getmtime(loc)


## some useful helper functions to consider
makefn('string') #uatomatically takes string, appends date + .csv

### masking
ins = (df.Lineage.str.contains("ins")) #TRUE when lineage contains ins
pbf = (df.Lineage.str.contains("PBF")) #TRUE when lineage contains PBF
insPBF = df[ins | pbf].copy()  # boolean operator -- include rows where either is TRUE

### helpful consolidators
makeConsolidations(df,
                   save = True,
                   popBack = False)  # creates two dataframes of all cages + dispensible % > .2, and either saves them independently or pops them back for  further manip

### helpful for euthanizing many animals
df.fillna("-").groupby(['Cage','DOD']).agg({'Tag':'unique', 'Lineage': 'unique'}).to_csv(fn)


In [3]:
## this is all the functions now contained in mouse_mgmt import
# # def importCleanDF(fn, sep = "\t",
#                   skip = 0,
#                   dropDeads = False,
#                   dropTransfers = True,
#                   dropHeaders = False,
#                   convertDOB = True,
#                   calcDispensables = True,
#                   stats=True):
#     '''skip rangi(0,9) for Harvard mastersheet
#     dropHeaders = True for Harvard Mas"'''
    
#     df = pd.read_csv(fn, sep = sep,
#                      skiprows = skip,
#                      parse_dates = True,
#                     dtype=str).dropna(axis = 0, how = 'all')
#     print("Read in CSV",fn) #logging
    
#     df.Sex = df.Sex.fillna("temp") 
#     print("Animals without gender given temp. gender") #if animals are not gendered
    
#     df.Cage = df.Cage.fillna('not included yet') #if animals are missing a cage
#     print("Animals without cage given temp. cage") #if animals are not gendered

#     df = parseDOD(df, drop = dropDeads)
    
#     if dropTransfers:
#         df = dropTransferred(df)
        
#     if dropHeaders:
#         df = df[df.Cage.str.contains("NRB|HAT|HOP",
#                                  case=False,
#                                  na=False)].reset_index(drop=True) #drop the header rows & reset the index
#         print("Retained rows with NRB, HAT, HOP, dropped otherwise drop headers")
        
#     if convertDOB:
#         df.DOB = cleanDOBs(df.DOB)
#     if calcDispensables:
#         df = calcDisp(df)
#     if df.Ear.any():
#         df.Ear =  df.Ear.apply(lambda row: "\'" + row if type(row)== str else np.nan) #convert ear tags to excel-clean formats
#         print("Parsed ear tags for Excel formatting")  
#     if stats:
#         colonyStats(df)
#     return df

# # def cleanDOBs(s):
# #     '''uses .map() to apply changes'''
# #     dates = {date:pd.to_datetime(date, errors = 'coerce') for date in s.unique()}
# #     return s.map(dates)
# # def parseDOD(df, drop = True):
# #     df['DOD'] =  df.DOD.apply(lambda row: row.strip().lower() if type(row)== str else np.nan) #convert ear tags to excel-clean formats
# #     df['parsedDOD'] = pd.to_datetime(df.DOD, errors = 'coerce')
# #     if drop:
# #         init = colonyStats(df)
# #         df = df[(~df.parsedDOD.notnull()) \
# #                 & (~df.DOD.str.contains('sperm|Diss|f.d.|mia.',
# #                                    na = False))] #drop assorted dead mice
# #         print("initial counts:", init, "\nafter dropping dead", colonyStats(df))
# #     return df



# #     return df      
# # def dropTransferred(df):
# #     init = colonyStats(df)
# #     df = df[~df.DOD.str.contains('transfer', na = False)]
# #     fin = colonyStats(df)
# #     print('after removing transferred', fin)
# #     return df
# # def collectAgeRange(df,
# #                     on_date,
# #                     oldest_requested_age,
# #                     youngest_requested_age,
# #                     lineage,
# #                     gender):
# #     early = pd.to_datetime(on_date) - pd.Timedelta(weeks = oldest_requested_age)
# #     late = pd.to_datetime(on_date) - pd.Timedelta(weeks = youngest_requested_age)
# #     mask = (df['DOB'] >= early)\
# #             & (df['DOB'] <= late)\
# #             & (df['Lineage'].str.contains(lineage))\
# #             & (df['Sex'].str.contains(gender))
# #     return early, late, mask
# # def colonyStats(df):
# #     '''helper function: returns number of mice, number of cages'''
# #     num_cages = len(df.Cage.unique())
# #     stats = {'numCages':len(df.Cage.unique()), 'numMice' : len(df)}
# #     if num_cages >=200:
# #         print("certainly cage reorg time")
# #     return stats
# # def makefn(string):
# #     '''returns string + formatted date to use as a file name'''
# #     return string + str(dt.today().strftime("_%Y-%B-%d")) + ".csv"
# # def tagSearch(string, df, column="Tag"):
# #     '''from a string, extracts all numbers and then searches for those numbers in a column'''
# #     find_t = re.findall(r'\d+', string)
# #     #search for that list in the spreadsheet and return only those rows
# #     return df[column].isin(find_t)
# # def makeOutputs(df, upcase=True):
# #     '''extremely specific to produce dispensible & sorted cage information'''
# #     fineGrainedDispense = df.query("Dispensible >= 0.2").sort_values(["Dispensible", "Cage"], ascending=False)
# #     cages = df.groupby("Cage").agg({'Dispensible':'first', #could be cleaner
# #                                     'Cage':'size', #yay!
# #                                     'Sex':'unique', #yay!
# #                                     'Color':'unique',
# #                                    'DOD':'unique'})   #yay!
# #     return fineGrainedDispense, cages
# # def calcTotals(df):
# #     counted = df.groupby("Cage").agg('size').reset_index(name='Total')
# #     return df.merge(counted)
# # def calcDisp(df):
# #     disp = df.groupby("Cage").DOD\
# #                 .value_counts(normalize=True,
# #                               dropna = False).unstack()
# #     if 'Dispensible' in disp:
# #         disp = disp.Dispensible.reset_index()
# #         print("Dispensible counts normalized + totals calculated")
# #         df = df.merge(disp)
# #     else:
# #         print("No dispensible mice, returning original DF with totals added")
# #     return calcTotals(df)

In [4]:
file = '05227_mastersheet - HopkinsColony.tsv'

In [5]:
ts = os.path.getmtime(file)
print("mtime:", dt.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

mtime: 2020-10-15 13:56:58


In [6]:
df = importCleanDF(file,
                   dropDeads=False,
                   dropTransfers=True)

Read in CSV 05227_mastersheet - HopkinsColony.tsv
Animals without gender given temp. gender
Animals without cage given temp. cage
Animals without specified lineage given NA
calculted total dispensible counts
calculated total cage sizes
calculated % disp
certainly cage reorg time
certainly cage reorg time
after removing transferred {'numCages': 200, 'numMice': 538}
Parsed ear tags for Excel formatting
certainly cage reorg time


In [9]:
df.groupby(['DOD']).agg({'Cage': 'nunique', 'DOD':'unique', 'Tag': 'unique'})

,Cage,DOD,Tag
DOD,,,
-,41,[-],"[5116, 5178, 5222, 5258, 5261, 5262, 5263, 526..."
07/17/20,1,[07/17/20],[5317]
1/20/2020,1,[1/20/2020],[2618]
1/3/2019,1,[1/3/2019],"[2617, 2619]"
10/08/20 f.d.,1,[10/08/20 f.d.],[5113]
11/23/2019,1,[11/23/2019],[2616]
2020/04/06 - f.d.,1,[2020/04/06 - f.d.],[5224]
3/12/2020,1,[3/12/2020],[2713]
3/17/2020,31,[3/17/2020],"[2463, 2633, 2622, 2625, 2620, 2632, 2623, 262..."


In [20]:
df.query('DOD == "breeding-putative-201015"')[easyCats].sort_values('DOB').to_csv(makefn('PB7_breeders'), sep = "\t")



In [85]:
# ## vickie cages
# # mutant:
# AA0105 AA0105
# AA0056 5171-5174 (WT / syn cre relevant)

23

In [9]:
frames = makeConsolidations(df, popBack=True, save=True)

Saved CSV with fns AllCages_2020-August-06.csv
Saved CSV with fns FemaleCons_2020-August-06.csv
returning 2 frames


In [10]:
easyCats.append('Genotype')

In [13]:
df.query('Cage.str.contains("AA0105|AA0056")')[easyCats].to_clipboard(index = False, sep = "\t")

In [26]:
df.query('Cage.str.contains("AA0105|AA0056")')[['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']].to_clipboard(index = False, sep = "\t")

In [4]:
mice = df.fillna("-").query('Tag.str.contains("5049|5253|5254|5242")')
cages = mice.Cage.unique()

In [40]:
# cages = mice.Cage.unique()
df[df.Cage.isin(cages)][easyCats].to_clipboard(index = False, sep = "\t")

['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage', 'Plate']

In [62]:
df[df.Cage.isin(cages)][['Tag', 'Cage', 'Ear', 'Sex', 'Color', 'DOB', 'Lineage']]

,Tag,Cage,Ear,Sex,Color,DOB,Lineage
36,5034,AA0010,'+/t,F,NaN,2019-12-31,fCas9 x hCD2
37,5035,AA0010,'++/t,F,NaN,2019-12-31,fCas9 x hCD2
38,5036,AA0010,'t/+,F,NaN,2019-12-31,fCas9 x hCD2
39,5048,AA0010,'+/t,F,Agouti,2019-12-30,1685ins21#2
40,5049,AA0010,'++/t,F,Agouti,2019-12-30,1685ins21#2
195,5242,AA0079,'+/t,F,Black,2020-02-05,1763PB#7
196,5243,AA0079,'++/t,F,Black,2020-02-05,1763PB#7
197,5298,AA0079,'+/t,F,Albino,2020-04-01,1763PB#7
198,5299,AA0079,'t/+,F,Albino,2020-04-01,1763PB#7
206,5253,AA0082,'+/t,F,Agouti,2020-02-11,1763PB#7


# code from summing total hgRNA counts for collecting animal samples

## splitting hgRNA genotypes into columns

In [4]:
## speciality speed code for re-splitting genotypes by hgRNA tyep
def splitGenotypes(df):
    split = df['Genotype'].str.split(r'(\w,)', expand=True)
    split[6] = split[6].str.extract('(\d+)')
    split = split.rename({
        0: "E",
        2: 'I',
        4: "S",
        6: 'ina'
    }, axis=1).drop(columns=[1, 3, 5])

    return split


split = splitGenotypes(df)
newcols = []
oldcols = []
for col in split:
    colN = str(col) + "_num"
    split[colN] = pd.to_numeric(split[col], errors='coerce')
    oldcols.append(col)
    newcols.append(colN)

split['summedhgRNAs'] = split[newcols].sum(axis=1)
split['summedhgRNAs_active'] = split[newcols[0:3]].sum(axis=1)
split = split.drop(oldcols[0:4], axis=1)
df = df.merge(split, left_index = True, right_index = True)

,Tag,Cage,Ear,Sex,Color,Name,Genotype,DOB,DOD,Father,...,parsedDOD,DispCount,CageSize,DispPercent,E_num,I_num,S_num,ina_num,summedhgRNAs,summedhgRNAs_active
6,2401,HOP0003,'+/+t,M,Albino,NaN,"5E,10I,20S,10ina",2019-05-28,"breeding - xcas9 f, experimental",1732,...,NaT,0.0,1,0.000000,5.0,10.0,20.0,10.0,45.0,35.0
7,2662,NRB1073-A,'-/ot,F,Gray,J024858-Cas9,J024858-Cas9,2019-08-16,"dispensible, check for pregnancies and euthani...",2065,...,NaT,2.0,2,1.000000,NaN,NaN,NaN,NaN,0.0,0.0
8,2548,NRB1073-A,'-/ot,M,Black,J024858-Cas9,J024858-Cas9,2019-07-16,dispensible,2065,...,NaT,2.0,2,1.000000,NaN,NaN,NaN,NaN,0.0,0.0
9,2663,NRB1073-B,'-/ot,F,Gray,J024858-Cas9,J024858-Cas9,2019-08-16,"dispensible, check for pregnancies and euthani...",2065,...,NaT,2.0,2,1.000000,NaN,NaN,NaN,NaN,0.0,0.0
10,2549,NRB1073-B,'-/ot,M,Black,J024858-Cas9,J024858-Cas9,2019-07-16,dispensible,2065,...,NaT,2.0,2,1.000000,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,5322,AA0106,'t/++,F,Agouti,NaN,NaN,2020-04-10,-,2550,...,NaT,0.0,5,0.000000,NaN,NaN,NaN,NaN,0.0,0.0
252,5323,AA0106,'notch-5,F,Agouti,NaN,NaN,2020-04-10,-,2550,...,NaT,0.0,5,0.000000,NaN,NaN,NaN,NaN,0.0,0.0
253,5324,AA0107,'+/t,F,Black,NaN,"fCas9 MT(+/+), synCre (+)",2020-03-24,breeder-backup,2728,...,NaT,1.0,3,0.333333,NaN,NaN,NaN,NaN,0.0,0.0
254,5325,AA0107,'++/t,F,Black,NaN,"fCas9 MT(+/+), synCre (+)",2020-03-24,breeder-jun01,2728,...,NaT,1.0,3,0.333333,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
# .to_clipboard(index=False, sep = "\t")
df.query('DOD.str.contains("zack")')[['Tag','Genotype', 'summedhgRNAs', 'summedhgRNAs_active']].to_csv('ZackTransferGenotypes.tsv', sep = '\t', index=False) 

## query each lineage + create CSV from it

In [ ]:
def query(df, lineage, disp=1):
    result = df.query(
        'Lineage.str.contains("{}")'.format(lineage))
    if disp:
        result = result.query('DispPercent >= {}'.format(disp))
    result = result.sort_values("summedhgRNAs_active", ascending=False)
    return result



frames = {}
lineages = ['#3', '#7', 'ins21', '-Cas9', 'PBF']

for line in lineages:
    frame = query(df, line).head(5)
    frames.update({line: frame})
for frame in frames:
    fn = makefn(frame)
    frames[frame].to_csv(fn, sep="\t")
    df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)
frame = query(df, 'fCas9').query('transgene.str.contains("-")')
frame = frame.sort_values(['fCas', 'transgene'], ascending = [True, True]).head(10)
fn = makefn('creFlipped-negative_ampliconSamples')
frame.to_csv(fn, sep ='\t')


# checking breedings against spreadsheet

In [ ]:
source_dir = "Breedings"
date = "2020-05"
fns = glob.glob(source_dir + '/*{}*'.format(date))
data = []

for fn in fns:
    frame = pd.read_csv(fn, sep="\t")
    frame['Lineage'], frame['Date'] = fn[10:24].split("-", 1)
    data.append(frame)

animals = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes

animals = animals.applymap(lambda x: x.strip("t") if isinstance(
    x, str) else x)  #remove t for easier comparison to big frame applys

tagCols = ['Male', 'Female1',
           'Female2']  #grab columns with animals for flattening
data = []
for col in tagCols:
    frame = pd.concat((animals[i] for i in [col, 'Lineage', 'Date']),
                      axis=1).rename({col: 'Tag'}, axis=1)
    data.append(frame)

#            #, animals['Lineage'], animals['Date']]
# #     data.append(frame)
breeders = pd.concat(
    data, ignore_index=True)  #dont want pandas to try an align row indexes;

## check that all animals are correctly marked in the spreadsheet

In [6]:
## #### parsing breedings etc - check that all breeders are marked as such on the sheet
breedrs = pd.merge(df, breeders, left_on ='TagClean', right_on = 'Tag', how = 'inner')
if len(breedrs.DOD.unique()) > 1:
    print("Not all breeders marked")
else:
    print("Only one DOD found, {}".format(breedrs.DOD.unique()[0]))
    
list(set(breeders['Tag'])-set(breedrs['TagClean']))

NameError: name 'breeders' is not defined

In [ ]:
##############################################

In [ ]:
# plate = df[plate].sort_values(['Column', 'Row'])
# #.sort_values(
#  #       ['Row',
#   #       'Column']).reset_index(drop=True)  # impt standard plate indexing

# plateNew = locs.merge(plate, how = 'outer')
# df = plateNew

# plateNew.pivot(index='Row', columns='Column',
#                       values=['Info'])

In [ ]:
# plate.Tag = df.Tag.apply(lambda x: "t" + str(x) if not x != x else 'empty')
# plate['Info'] = ['\n'.join([str(x), str(y)]) for x, y in zip(plate['Tag'], plate['Lineage'])]



# assessing parental genotypes to consider offspring (for fCas9 maintenance/breeding)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
fathers = df[df.Lineage.str.contains('fCas9')].Father.unique().flatten()
mothers = df[df.Lineage.str.contains('fCas9')].Mother.unique().flatten()
rents = np.append(fathers, mothers)
parentList = []
for i in rents:
    parentList.extend(i.split("/")) 
parents = df.loc[df['Tag'].isin(parentList)]

In [ ]:
## easy categories
plateCats =  ['Genotype','PlateID', 'Row', 'Column',
       'Position']
cats = easyCats + plateCats

In [ ]:
df[['fCas', 'transgene']] = df['Genotype'].str.split(",", 1, expand = True)

In [ ]:
df.Lineage = df.Lineage.fillna("--")
linMask = df.Lineage.str.contains('fCas9')
genoMask = df.Genotype.isnull()
fCas9 = df[linMask].fillna("-")
H_linMask = fCas9.Lineage.str.contains('hCD2')
S_linMask = fCas9.Lineage.str.lower().str.contains('syn')

In [ ]:
fCas9.groupby(['Sex', 'DOD', 'fCas', 'transgene']).agg({'Genotype': 'count',
                                                   'Tag':'unique', 'Cage':'unique', 
                                                  'DOB': ['min', 'max']})

In [ ]:
cages = df.groupby(["Cage", "Sex"]).agg({'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

In [ ]:
df.groupby('Cage').agg({
    'dispensible': 'mean',
    'Cage': 'size',
    'Sex': 'unique',
    'DOD': 'unique',
    'Tag': 'unique',
    'Color': 'unique',
    'Ear': 'unique',
    'Lineage': 'unique'
}).sort_values('dispensible', ascending = False)

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage': 'size',
                                         'DOD': 'unique',
                                         'Tag':'unique',
                                        'Color' : 'unique',
                                         'Ear' : 'unique',}).sort_values

In [ ]:
df[df.Sex == "F"].groupby(['Cage']).agg({'Cage':'size',
                                         'Color':'unique',
                                         'DOD':'unique',
                                         'Lineage':'unique'})   #yay!

# retrieve animals with null genotypes in a certain lineage + get their well plates

In [ ]:
insGeno = df[(df.Lineage.str.contains("ins21")) & (df.Genotype.isnull())]

In [ ]:
locs = pd.read_pickle('96-wellLocs.pkl').sort_values(
        ['Column',
         'Row']).reset_index(drop=True)  # impt standard plate indexing
# locs.join(df, rsuffix = "_init")
# insGeno.sort_values(join(locs, rsuffix = 'init')
locs.join(insGeno, rsuffix='init')

In [ ]:
insGeno.pivot(index='WellRow', columns='WellCol',
                      values=['Tag', 'Plate'])

# generate a general colony pivot table by lineage, sex, and DOB with total numbers

In [ ]:
saveFn = makefn(file.replace(" ", "")+'_summary')

remapDict= {'ins': '1685ins21#2',
            'pb3' : '1763PB#3',
            'pb7' : '1763PB#7',
            'pbf': '1853PBF#1'}
for key, value in remapDict.items():
    df.Lineage = df.Lineage.apply(lambda row: value if key in row.lower() else row)
grouped = df.groupby(['Lineage',
            'Sex',
            pd.Grouper(key='DOB',freq='q')]).size().unstack('DOB').reset_index()
grouped['Total'] = grouped.sum(axis=1)
grouped.fillna("-").to_csv(saveFn)

In [ ]:
saveFn = makefn(file.replace(" ", "")+'_femaleConsolidation')

In [ ]:
df.groupby(["Cage"]).agg({'Cage': 'count',  'Sex': 'unique',
                                 'Color':'unique',
                                'DOD': 'unique'})

In [ ]:
mating = tagSearch(ins + PB3 + PB7 + PBF + cre)
df[mating].sort_values('Lineage').to_csv('file.csv')

# break - 2019 and previous code snippets

In [ ]:
CreMask = (df.Lineage.str.contains("fCas", na=False))
PBFMask = (df.Lineage.str.contains("PBF", na=False))
insMask = tagSearch("t2620 t2632 t2623 t2463 t2625 t2622 t2616 t2626 t2633 t2720 t2721 t2722 t2755 t2750 t2751 t2415 t2445")
alreadyShipped = (df.Cage.str.contains("HOP", na= True))
traveling = (df.DOD.str.endswith("hopkins| ", na = False))

In [ ]:
shipped_2 = df[(df.DOD.str.endswith("hopkins", na = False))]

In [ ]:
shipped_2.sort_values(["Cage", "Tag"])[['Cage', 'Sex', 'Tag', 'Ear', 'Color', 'DOB']].to_clipboard(excel=True, index=False)

In [ ]:
shipped_2.groupby(["Cage"]).agg({'Cage': 'count', 'Sex':'unique'}).to_clipboard(excel=True)

In [ ]:
hop = df[((traveling & (CreMask | PBFMask)) | insMask ) & ~alreadyShipped]

In [ ]:
hop.groupby(["Cage", "Sex"]).agg({'Cage': 'count'})

In [ ]:
hop.groupby(["Sex", "Cage", "Tag"]).agg({'Lineage': 'unique',
                                         'Color': 'unique',
                                        'Ear': 'unique'})

In [ ]:
hop.groupby(['Sex','Cage']).agg({'Cage':'count'})

In [ ]:
hop.sort_values(['Sex', "Lineage", 'Tag'])

In [ ]:
df[(df.Genotype.isnull()) \
   & (df['DOD'] != "Dispensible")].fillna("x").groupby(["Lineage", 'Cage']).agg({'Sex': 'count',
                                                                                   'Tag':'unique',
                                                                                   'Name': 'unique' })

In [ ]:
#mask for sorting centrifuge tube samples
freezerMask = (df.Tag.notnull()) & \
    (~df['Notebook info'].str.startswith("T|Plate", na=False))&\
     (df.Lineage != 'J024858-Cas9')

#and the results
df[freezerMask].sort_values("Tag")

In [ ]:
earlyMask, lateMask, fMask = collectAgeRange(df, '2019-12-01',
                                             28, 0 , "", "F")
earlyMask, lateMask, mMask = collectAgeRange(df, '2019-12-01',
                                             40, 0 , "", "M")

inc = df[mMask].append(df[fMask]) #combine

inc['DOD'], inc['parsedDOD'] =  zip(*inc.DOD.apply(lambda row: cleanDOD(row))) #parse DOD
inc = inc[(~inc.parsedDOD.notnull()) \
          & (~inc.DOD.str.contains('sperm|Diss|f.d.',
                                   na = False))] #drop assorted dead mice

In [ ]:
inc.fillna('n/a').groupby(['Lineage',
                           'DOD',
                           'Sex']).agg({'DOB': ['count',
                                                'min',
                                                'max']})

In [ ]:
inc[inc.DOD.str.contains('Hopkins', na= False)].groupby("Sex").agg({'Sex': 'count',
                                                    'Cage': 'nunique'})


In [ ]:
noMice, noCages

In [ ]:
df[df.Plate == "T885765"].sort_values(['WellRow',
                                       'WellCol'])[['Position',
                                                    'Tag',
                                                    'Lineage']].to_clipboard(excel=True, header = False, index = False)

In [ ]:
def is_coherent(seq):
    return seq == range(seq[0], seq[-1]+1)

In [ ]:
MMRRC_cats = ['Tag',
           'Cage',
           'Ear',
           'Sex',
           'Color',
           'Genotype',
           'DOB',
           'Lineage']

In [ ]:
# earlyMask, lateMask, mMask = collectAgeRange(df,
#                                              '2019-09-16',
#                                              13, 8,
#                                              "PB#", "M")
# earlyMask, lateMask, fMask = collectAgeRange(df, 
#                                              '2019-09-16',
#                                              4, 1,
#                                              "PB#", "F")
# shippedMask = (df.DOD.str.contains("mrrc|opkins",
#                                    na=False)) & (df.Lineage.str.contains("PB#")) #PBs marked for any maintenance movement

# df[mMask|fMask|shippedMask].groupby(["Lineage",
#                                      "Sex",
#                                      "DOD"]).agg({'Tag':'count',
#                                                    'DOB': ['first', 'last']}) #

dispense, cSorted = makeOutputs(df)
cSorted.to_clipboard(excel=True, index = True)

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-09-16', 4, 1, "", "F")
df[incMask][['Tag',
             'Cage',
             'Ear',
             'Sex',
             'Color',
             'DOB',
             'Genotype',
             'Lineage']].groupby("Lineage").count()

In [ ]:
earlyMask, lateMask, incMask = collectAgeRange(df, '2019-10-04', 10, 0 , "", "")
#inc = df[incMask]
#inc = inc.fillna("BLANK")

# [['Tag',
#              'Cage',
#              'Ear',
#              'Sex',
#              'Color',
#              'DOB',
#              'Genotype',
#              'Lineage']]

In [ ]:
# df[df.DOD == 'Shipped to mmrrc on 9/10'][['Tag',
#              'Color',
#              'Ear',
#              'Cage', 
#              'Sex',
#              'Genotype',
#              'DOB',
#              'Lineage']].sort_values(by = ['Cage',
#                                            'Tag'])\
#                 .fillna("missing").to_clipboard(index = False,
#                                                excel = True)

In [ ]:
# ## MICE WITH UNKNOWN FATE
# df[(df.DOD.isnull())].groupby(["Lineage", "Cage"]).agg({'DOB': ['max', 
#                                                              'min', 
#                                                              'count'],
#                                                         'Sex': 'max',
#                                                         'Name': 'unique'}).fillna("")

In [ ]:
########## selecting mice by age#########
shipDate = '2019-09-16'
early1, late1, mask1 = collectAgeRange(df, shipDate, 13, 8, "PB#", "M")
early2, late2, mask2 = collectAgeRange(df, shipDate, 9, 6.5, "PB#", "F")

males_20190916  = df[mask1].sort_values(["Lineage", "Genotype"])
females_20190916 = df[mask2].sort_values(["Lineage", "Genotype"])

shipment_20190916 = males_20190916.append(females_20190916)

# Sep09_shipment.sort_values(["Lineage",
#                             "Sex",
#                             "Cage"]).to_clipboard(excel=True,
#                                                   index = False)

# Sep09_shipment.groupby(["Lineage",
#                         "Sex",
#                         "Cage"])['Tag',
#                                  'Genotype'].count().to_clipboard()

In [ ]:
plate_reseq.sort_values(["Tag"])[['Tag',
                                  'Lineage',
                                  'Plate',
                                  'WellRow',
                                  'WellCol']].to_clipboard(excel = True, index= False)

In [ ]:
df['Tag'] = 't' + df['Tag'].astype(str)

In [ ]:
KL14_plateB = df[df['Plate'].str\
   .contains("T885773|T885761", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
KL14_plateB[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
KL14_plateA = df[df['Plate'].str\
   .contains("T885772", na= False)]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol']).reset_index(drop = True)

In [ ]:
df[df['Plate'].str\
   .contains("T88572|T885761|T885772|T885773",
             na = False)][['Tag', 'Plate', 'Well']]\
    .sort_values(by=['Plate', 'Well'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
KL14_plateA[['Tag','Lineage','Plate', 'WellRow', 'WellCol']]\
    .to_clipboard(excel=True, index=False)

In [ ]:
#KL14_plateA.reindex('WellCol', method='ffill')

In [ ]:
KL14_plateA.groupby('WellRow').Lineage.count()

In [ ]:
KL14_plateA

In [ ]:
KL14_plateA[['Tag', 'Plate', 'WellRow', 'WellCol']]\
    .sort_values(by=['Plate', 'WellRow', 'WellCol'])\
    .to_clipboard(excel=True, index= False)

In [ ]:
## lineage stats over time
df.groupby([pd.Grouper(key="DOB", freq = "M"),
            "Lineage"]).Cage.count().unstack('DOB')

In [ ]:
def dfContains(string, column = "Cage", keep_empties = False, case = False, df = df):
    '''given a cage number/s (formatted as "nrbxxxx|nrbxxx") etc, the indexes of entries where true'''
    return df[column].str.contains(string, na = keep_empties, case = case)

In [ ]:
df[df.DOD.str.contains("Sperm", na = False)].groupby('Lineage').count()

In [ ]:
df[(dfContains("1036|1037"))
   & (df.Sex == "M")][['Tag',
                      'Cage',
                       'Ear',
                       'Sex',
                       'DOB',
                       'Notes']]

In [ ]:
PB = ['1763PB#7', '1763PB#3']
counted = df.groupby("Cage").size().reset_index(name = 'Total')
df = df.merge(counted)
PB_solo_males = df[(df.Lineage.isin(PB))&(df.Sex == "M")&(df.Total == 1)]